In [2]:
!pip install --upgrade --quiet langchain pinecone-client pinecone-text openai


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + C:\Users\arpan\OneDrive\Documents\GitHub\Updated-Langchain\.venv\Scripts\python.exe C:\Users\arpan\AppData\Local\Temp\pip-install-nq4kv_9c\numpy_839b35b77a334e2a805e9ffa0740109d\vendored-meson\meson\meson.py setup C:\Users\arpan\AppData\Local\Temp\pip-install-nq4kv_9c\numpy_839b35b77a334e2a805e9ffa0740109d C:\Users\arpan\AppData\Local\Temp\pip-install-nq4kv_9c\numpy_839b35b77a334e2a805e9ffa0740109d\.mesonpy-vtrfctgt -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\arpan\AppData\Local\Temp\pip-install-nq4kv_9c\numpy_839b35b77a334e2a805e9ffa0740109d\.mesonpy-vtrfctgt\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\arpan\AppData\Local\Temp\pip-install-nq4kv_9c\numpy_839b35b77a334e2a805e9ffa0740109d
      Build dir: C:\Users\arpan\AppData\Local\Te

In [6]:
import pinecone
from pinecone import ServerlessSpec
import os

# Initialize Pinecone
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment="us-east-1-aws")

# Define index name and dimensions
index_name = "hybrid-search-index"
dimension = 1536  # Adjust based on your embedding model

# Create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=dimension,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pinecone.Index(index_name)

import os
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from pinecone_text.sparse import BM25Encoder

# Initialize embedding model
embedding_model = OpenAIEmbeddings()

# Initialize BM25 encoder
bm25_encoder = BM25Encoder().default()

# Fit BM25 encoder on your corpus
corpus = ["Document 1 text", "Document 2 text", "Document 3 text"]
bm25_encoder.fit(corpus)

# Optionally, save and load BM25 values
bm25_encoder.dump("bm25_values.json")
bm25_encoder = BM25Encoder().load("bm25_values.json")

from langchain.retrievers import PineconeHybridSearchRetriever

# Initialize the retriever
retriever = PineconeHybridSearchRetriever(
    embeddings=embedding_model,
    sparse_encoder=bm25_encoder,
    index=index
)

documents = ["Document 1 text", "Document 2 text", "Document 3 text"]
retriever.add_texts(documents)

query = "Your search query here"
results = retriever.invoke(query)

# Display the results
for doc in results:
    print(doc.page_content)

KeyError: 'PINECONE_API_KEY'